In [1]:
import logging
logging.getLogger().setLevel(logging.INFO)
import numpy as np
import pandas as pd
from utils.utils import PROJECT_DATA_DIR
import os
import mxnet as mx
import mxnet.ndarray as nd
from time import time
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from load_preprocess import (load_data,
                             get_xy,
                             scale_data,
                             binarize_y,
                             prepare_data)
import warnings
warnings.filterwarnings("ignore",  category=DeprecationWarning)
from DL_mxnet_symbol import train_dnn, train_lstm

C:\Users\gilbe\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = load_data(file='all_training_400_minisensor_1.csv')
test = load_data(file='all_test_400_minisensor.csv')
xtrain, ytrain, xtest, ytest = prepare_data(train, test, binary_class=True)
xtrain_sc, xtest_sc = scale_data(xtrain, xtest)

In [3]:
(ytrain.shape, ytest.shape)

((91104,), (22776,))

In [4]:
xtrain_mx = mx.nd.array(xtrain_sc, dtype=np.float32)
ytrain_mx = mx.nd.array(ytrain.reshape(-1, 1))
xtest_mx = mx.nd.array(xtest_sc, dtype=np.float32)
ytest_mx = mx.nd.array(ytest.reshape(-1, 1))
batch_size=2**9

train_iter = mx.io.NDArrayIter(
    xtrain_mx,
    ytrain_mx,
    batch_size=batch_size,
    shuffle=True)

val_iter = mx.io.NDArrayIter(
    xtest_mx,
    ytest_mx,
    batch_size=batch_size)


In [5]:
train_dnn(train_iter, val_iter)

INFO:root:Epoch[0] Batch [100]	Speed: 131102.41 samples/sec	accuracy=1.000000
INFO:root:Epoch[0] Train-accuracy=1.000000
INFO:root:Epoch[0] Time cost=0.844
INFO:root:Epoch[0] Validation-accuracy=1.000000
INFO:root:Epoch[1] Batch [100]	Speed: 113010.47 samples/sec	accuracy=1.000000
INFO:root:Epoch[1] Train-accuracy=1.000000
INFO:root:Epoch[1] Time cost=0.844
INFO:root:Epoch[1] Validation-accuracy=1.000000
INFO:root:Epoch[2] Batch [100]	Speed: 109252.28 samples/sec	accuracy=1.000000
INFO:root:Epoch[2] Train-accuracy=1.000000
INFO:root:Epoch[2] Time cost=0.828
INFO:root:Epoch[2] Validation-accuracy=1.000000
INFO:root:Epoch[3] Batch [100]	Speed: 113011.48 samples/sec	accuracy=1.000000
INFO:root:Epoch[3] Train-accuracy=1.000000
INFO:root:Epoch[3] Time cost=0.797
INFO:root:Epoch[3] Validation-accuracy=1.000000
INFO:root:Epoch[4] Batch [100]	Speed: 112661.38 samples/sec	accuracy=1.000000
INFO:root:Epoch[4] Train-accuracy=1.000000
INFO:root:Epoch[4] Time cost=0.814
INFO:root:Epoch[4] Validatio

## LSTM

In [6]:
xtrain_lstm = xtrain.values.reshape(-1, 3)
xtest_lstm = xtest.values.reshape(-1, 3)
scaler = QuantileTransformer(output_distribution='normal')
xtrain_lstm_sc = scaler.fit_transform(xtrain_lstm)
xtest_lstm_sc = scaler.transform(xtest_lstm)

print(xtrain_lstm_sc.shape)
print(xtest_lstm_sc.shape)
""" Change time steps from 400 to 20 to test if this is the problem"""
xtrain_lstm_sc = mx.nd.array(xtrain_lstm_sc.reshape(-1, 400, 3))
val_lstm_sc = mx.nd.array(xtest_lstm_sc.reshape(-1, 400, 3))
print('shape of xtrain_lstm_sc:', xtrain_lstm_sc.shape)

train_lstm_iter = mx.io.NDArrayIter(
    xtrain_lstm_sc,
    ytrain_mx,
    batch_size,
    shuffle=True,
    last_batch_handle='discard')

val_lstm_iter = mx.io.NDArrayIter(
    val_lstm_sc,
    ytest_mx,
    batch_size,
    shuffle=False,
    last_batch_handle='discard')

(36441600, 3)
(9110400, 3)
shape of xtrain_lstm_sc: (91104, 400, 3)


In [7]:
## This line is erroneuous
#train_lstm(train_lstm_iter, val_lstm_iter)

In [8]:
#stacked_rnn_cells = mx.rnn.SequentialRNNCell()
#stacked_rnn_cells.add(mx.rnn.)

In [15]:
def rnn_fused(timesteps=400, 
              num_layers=3, 
              mode='lstm', 
              num_hidden=20,
              dropout=0.4,
              num_outputs=1, 
              batch_size=2**9, 
              input_dim=3):
    
    data = mx.sym.Variable('data')
    """ Reshape input """
    input_shape = (timesteps, batch_size, input_dim)
    data = mx.sym.Reshape(data, shape=input_shape)
    
    """num_hidden: number of units in output symbol"""
    for i in range(num_layers):
        fused_lstm_cell = mx.rnn.FusedRNNCell(
            num_hidden=num_hidden, 
            dropout=dropout)
        """ Implement many layers with for-loop as it is
        more effective when using multiple gpus"""
        outputs, _  = fused_lstm_cell.unroll(
            length=timesteps, 
            inputs=data, 
            merge_outputs=True)
        """ Reshape output from LSTM"""
    output_shape = (batch_size, timesteps, num_hidden)
    outputs = mx.sym.Reshape(outputs, shape=output_shape)
    outputs = mx.sym.FullyConnected(
        data=outputs, 
        name='out', 
        num_hidden=num_outputs)
    outputs = mx.sym.LogisticRegressionOutput(
        outputs,
        name='softmax')
    
    return outputs

In [16]:
net = rnn_fused()

mod = mx.mod.Module(net, context=mx.gpu())
mod.bind(data_shapes=train_lstm_iter.provide_data, 
        label_shapes=train_lstm_iter.provide_label)
mod.init_params(initializer=mx.init.Xavier())
mod.init_optimizer(
            optimizer='sgd',
optimizer_params=(('learning_rate', 0.01), ))

mod.fit(train_data=train_iter,
        eval_data=val_iter,
        #optimizer='s',
        #optimizer_params={'learning_rate': 0.01},
        eval_metric='acc',
        num_epoch=20,
        batch_end_callback = mx.callback.Speedometer(batch_size, 100),
)

INFO:root:Epoch[0] Batch [100]	Speed: 18947.86 samples/sec	accuracy=1.000000
INFO:root:Epoch[0] Train-accuracy=1.000000
INFO:root:Epoch[0] Time cost=4.753
INFO:root:Epoch[0] Validation-accuracy=1.000000
INFO:root:Epoch[1] Batch [100]	Speed: 19670.24 samples/sec	accuracy=1.000000
INFO:root:Epoch[1] Train-accuracy=1.000000
INFO:root:Epoch[1] Time cost=4.618
INFO:root:Epoch[1] Validation-accuracy=1.000000
INFO:root:Epoch[2] Batch [100]	Speed: 19744.14 samples/sec	accuracy=1.000000
INFO:root:Epoch[2] Train-accuracy=1.000000
INFO:root:Epoch[2] Time cost=4.615
INFO:root:Epoch[2] Validation-accuracy=1.000000
INFO:root:Epoch[3] Batch [100]	Speed: 19504.09 samples/sec	accuracy=1.000000
INFO:root:Epoch[3] Train-accuracy=1.000000
INFO:root:Epoch[3] Time cost=4.679
INFO:root:Epoch[3] Validation-accuracy=1.000000
INFO:root:Epoch[4] Batch [100]	Speed: 19734.23 samples/sec	accuracy=1.000000
INFO:root:Epoch[4] Train-accuracy=1.000000
INFO:root:Epoch[4] Time cost=4.613
INFO:root:Epoch[4] Validation-acc

In [11]:
# Define new model
ctx = mx.gpu()
def rnn(timesteps=400, 
        nhid=10, 
        num_layers=1, 
        dropout=0.4, 
        batch_size=2**9, 
        num_outputs=1, 
        features=3):
    
    data = mx.sym.Variable('data')
    weight = mx.sym.Variable('encoder_weight', 
                             init=mx.init.Uniform(0.1))

    states = []
    state_names = []
    outputs = mx.sym.Dropout(data, p=dropout)
    
    for i in range(num_layers):
        prefix = 'lstm_l%d_' % i
        cell = mx.rnn.FusedRNNCell(num_hidden=nhid, 
                                   prefix=prefix,
                                   get_next_state=True, 
                                   forget_bias=0.0,
                                   dropout=dropout)
        
        state_shape= (1, batch_size, nhid)#(batch_size, timesteps, features)
        
        begin_cell_state_name = prefix + 'cell'
        begin_hidden_state_name = prefix + 'hidden'
    
        begin_cell_state = mx.sym.var(begin_cell_state_name, 
                                      shape=state_shape)
       
        begin_hidden_state = mx.sym.var(begin_hidden_state_name, 
                                        shape=state_shape)
        state_names += [begin_cell_state_name, begin_hidden_state_name]
        
        outputs, next_states = cell.unroll(timesteps, 
                                           inputs=outputs, 
                                           begin_state=[begin_cell_state, 
                                                        begin_hidden_state],
                                           merge_outputs=True, 
                                           layout='TCN')
        
        outputs = mx.sym.Dropout(outputs, p=0.4)
        states += next_states
    outputs = mx.sym.Reshape(outputs, shape=(-1, nhid))
    
    outputs = mx.sym.FullyConnected(data=outputs, name='out', 
                                    num_hidden=num_outputs)
    
    outputs = mx.sym.LogisticRegressionOutput(outputs,
                                              name='softmax')

    
    return outputs, [mx.sym.stop_gradient(s) for s in states], state_names
    
    

def softmax_ce_loss(pred):
    # softmax cross-entropy loss
    label = mx.sym.Variable('label')
    label = mx.sym.Reshape(label, shape=(-1,))
    logits = mx.sym.log_softmax(pred, axis=-1)
    loss = -mx.sym.pick(logits, label, axis=-1, keepdims=True)
    loss = mx.sym.mean(loss, axis=0, exclude=True)
    return mx.sym.make_loss(loss, name='nll')


In [12]:
from module import CustomStatefulModule

In [13]:

#pred, states, state_names = rnn()
#loss = softmax_ce_loss(pred)
#module = CustomStatefulModule(loss, 
#                              states, 
#                              state_names=state_names, 
#                              context=ctx)
#module.bind(data_shapes=train_lstm_iter.provide_data, 
#            label_shapes=train_lstm_iter.provide_label)
#module.init_params(initializer=mx.init.Xavier())
#optimizer = mx.optimizer.create('sgd', 
#                                learning_rate=0.01, 
#                                rescale_grad=1.0/batch_size)
#module.init_optimizer(optimizer=optimizer)

    

512